# Práctica 7: Modelos del lenguaje

## Preprocesamiento

### Leer archivo y generar lista de oraciones

In [93]:
path = './el-quijote.txt'

In [94]:
import ast
encoding = 'utf-8-sig'

try:
    with open(path, 'r', encoding=encoding) as f:
        contents = f.readlines()
        #parsed_corpus = [ ast.literal_eval(x) for x in contents ]
except UnicodeDecodeError:
    print("Error: Unable to decode the file with the specified encoding.")
    
len(contents)

35522

In [95]:
contents[10:20]

['podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de\n',
 'complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo\n',
 'de la caza. Quieren decir que tenía el sobrenombre de Quijada, o Quesada,\n',
 'que en esto hay alguna diferencia en los autores que deste caso escriben;\n',
 'aunque, por conjeturas verosímiles, se deja entender que se llamaba\n',
 'Quejana. Pero esto importa poco a nuestro cuento; basta que en la narración\n',
 'dél no se salga un punto de la verdad.\n',
 '\n',
 'Es, pues, de saber que este sobredicho hidalgo, los ratos que estaba\n',
 'ocioso, que eran los más del año, se daba a leer libros de caballerías, con\n']

In [118]:
from functools import reduce
import operator

def get_sentences(corpus):
    """
    Recibe una lista de líneas que terminan en '\n', y genera otra lista que contiene el mismo
    texto, pero con '\n\n' y '.' como separador.
    También se deshace de todos los '...'
    """
    parsed_corpus = [ '<linejump>' if x == '\n' else x[:-1] for x in contents ]
    parsed_corpus = ' '.join(parsed_corpus).split('<linejump>')
    parsed_corpus = [x.strip() for x in parsed_corpus]
    parsed_corpus = [ x.replace('...','').replace('.', '.<eos>').split('<eos>') for x in parsed_corpus ]
    parsed_corpus = reduce(operator.concat, parsed_corpus)
    parsed_corpus = filter(lambda x: len(x)>0, parsed_corpus)
    parsed_corpus = [x.strip() for x in parsed_corpus]
    return parsed_corpus

In [121]:
corpus_sentences = get_sentences(contents)
corpus_sentences[:10]

['En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor.',
 'Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda.',
 'El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino.',
 'Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera.',
 'Frisaba la edad de nuestro hidalgo con los cincuenta años; era de complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza.',
 'Quieren decir que tenía el sobrenombre de Quijada, o Quesada, que en esto hay alguna difer

### Preprocesar oraciones

Pasar todo a minúsculas, y eliminar algunos signos de puntuación (guión largo, comillas, y comillas españolas).

In [125]:
from nltk.tokenize import word_tokenize

def delete_substrings(string: str, chars: list) -> str:
    final_str = string
    for char in chars:
        final_str = final_str.replace(char, '')
    return final_str

def process_sentences(sentences: list[str], stopwords: list) -> list:
    clean_sentences = [delete_substrings(sent.lower(), stopwords) for sent in sentences]
    # Para que la tokenización funcione necesitamos que los signos de puntuación estén separadas de las palabras.
    # Es decir, algo como "palabra," debe estar como "palabra ,"
    # Haremos esto con el tokenizer de nltk
    processed_sentences = [' '.join(word_tokenize(t)) for t in clean_sentences]
    return processed_sentences

In [126]:
stopwords = ["''","—","«","»"]
corpus_sentences = process_sentences(corpus_sentences, stopwords)

In [127]:
corpus_sentences[:10]

['en un lugar de la mancha , de cuyo nombre no quiero acordarme , no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero , adarga antigua , rocín flaco y galgo corredor .',
 'una olla de algo más vaca que carnero , salpicón las más noches , duelos y quebrantos los sábados , lantejas los viernes , algún palomino de añadidura los domingos , consumían las tres partes de su hacienda .',
 'el resto della concluían sayo de velarte , calzas de velludo para las fiestas , con sus pantuflos de lo mesmo , y los días de entresemana se honraba con su vellorí de lo más fino .',
 'tenía en su casa una ama que pasaba de los cuarenta , y una sobrina que no llegaba a los veinte , y un mozo de campo y plaza , que así ensillaba el rocín como tomaba la podadera .',
 'frisaba la edad de nuestro hidalgo con los cincuenta años ; era de complexión recia , seco de carnes , enjuto de rostro , gran madrugador y amigo de la caza .',
 'quieren decir que tenía el sobrenombre de quijada , o quesada , qu

### Tokenizar

Vamos a usar subword tokenization, como se muestra en el notebook de la práctica 5.

In [104]:
!pip install subword-nmt

  Obtaining dependency information for mock from https://files.pythonhosted.org/packages/6b/20/471f41173930550f279ccb65596a5ac19b9ac974a8d93679bcd3e0c31498/mock-5.1.0-py3-none-any.whl.metadata


In [193]:
# Entrenamos un tokenizador con BPE usando subword-nmt.
# Para esto necesitamos ingresarle el texto plano del corpus:

def write_plain_text_corpus(raw_text: str, file_name: str) -> None:
    with open(f"{file_name}.txt", "w", encoding="utf-8") as f:
        f.write(raw_text)

plain_corpus = '\n'.join(corpus_sentences)
write_plain_text_corpus(plain_corpus, "plain_processed_quijote")

In [194]:
from collections import Counter

# Primero checamos de qué tamaño es nuestro vocabulario
print("tokens:", len(plain_corpus.split()))
types_in_corpus = Counter(plain_corpus.split())
print("types in corpus:", len(types_in_corpus))

tokens: 425913
types in corpus: 22653


In [195]:
types_in_corpus.most_common(15)

[(',', 39209),
 ('que', 20079),
 ('y', 17688),
 ('de', 17540),
 ('la', 10003),
 ('a', 9565),
 ('en', 7967),
 ('el', 7933),
 ('.', 7715),
 ('no', 6126),
 (';', 4709),
 ('los', 4630),
 ('se', 4549),
 ('con', 4053),
 ('por', 3785),
 ('las', 3381),
 ('lo', 3347),
 ('le', 3331),
 ('su', 3273),
 ('don', 2547),
 ('del', 2370),
 ('me', 2298),
 ('como', 2215),
 ('sancho', 2118),
 ('quijote', 2078),
 ('es', 2074),
 ('yo', 2041),
 (':', 2009),
 ('más', 1983),
 ('si', 1914),
 ('un', 1885),
 ('dijo', 1801),
 ('mi', 1671),
 ('al', 1669),
 ('para', 1393),
 ('porque', 1377),
 ('ni', 1328),
 ('una', 1308),
 ('él', 1258),
 ('tan', 1206)]

In [196]:
!subword-nmt learn-bpe -s 2000 < plain_processed_quijote.txt > quijote_tokenizer.model


100%|##########| 2000/2000 [00:05<00:00, 348.05it/s]


In [251]:
tokenized = !echo "caminando nuestro flamante aventurero" | subword-nmt apply-bpe -c quijote_tokenizer.model
print("tokenized:", list(tokenized)[0].replace('"',''))

tokenized: @@ camin@@ ando nuestro fl@@ am@@ ante aventu@@ r@@ er@@ o@@  


In [198]:
!subword-nmt apply-bpe -c quijote_tokenizer.model < plain_processed_quijote.txt > quijote_tokenized_2k.txt

In [238]:
# Ahora, con esta tokenización, veremos cuántos tokens y tipos hay

with open("quijote_tokenized_2k.txt", "r", encoding="utf-8") as f:
    tokenized_text = f.read()

In [239]:
# Agregamos EOS y BOS
tokenized_text = ['<s> ' + sent + ' </s>' for sent in tokenized_text.split('\n')]
tokenized_text = '\n'.join(tokenized_text).replace('.','')

In [240]:
tokenized_corpus = tokenized_text.split()

In [241]:
print("tokens:", len(tokenized_corpus))
types_in_corpus = Counter(tokenized_corpus)
print("types in corpus:", len(types_in_corpus))

tokens: 574450
types in corpus: 2062


In [242]:
types_in_corpus.most_common(15)

[(',', 39209),
 ('que', 20203),
 ('de', 18019),
 ('y', 17784),
 ('la', 10723),
 ('a', 10677),
 ('<s>', 9248),
 ('</s>', 9248),
 ('en', 8656),
 ('el', 8103),
 ('no', 6651),
 ('los', 5103),
 ('se', 5073),
 (';', 4709),
 ('con', 4059)]

Ahora sí, formamos el corpus con las oraciones tokenizadas

In [304]:
tokenized_corpus_sentences = [ sent.split() for sent in tokenized_text.split('\n') ]

In [305]:
print(tokenized_corpus_sentences[0])

['<s>', 'en', 'un', 'lugar', 'de', 'la', 'mancha', ',', 'de', 'cuyo', 'nombre', 'no', 'quiero', 'acor@@', 'darme', ',', 'no', 'ha', 'mucho', 'tiempo', 'que', 'vi@@', 'vía', 'un', 'hidalgo', 'de', 'los', 'de', 'lanza', 'en', 'as@@', 'ti@@', 'll@@', 'ero', ',', 'ad@@', 'ar@@', 'ga', 'anti@@', 'gua', ',', 'ro@@', 'c@@', 'ín', 'f@@', 'lac@@', 'o', 'y', 'gal@@', 'go', 'corre@@', 'dor', '</s>']


### Diccionario de vocabulario
Nuestro modelo del lenguaje requiere que pasemos nuestras palabras a indices numericos. Utilizaremos enteros para estimar el modelo. 

Crearemos dos diccionarios: 
1. el primero tomara la palabra y lo convertira a indice (Para acceder a las probabilidades del modelo) 
2. El segundo tomará los indices y los convertira de vuelta a palabras (Nos ayudará a recuperar las palabras a partir de los índices del modelo)

In [306]:
vocabulary = list(types_in_corpus.keys())
vocabulary_by_type = { tipo: indice for (tipo, indice) in zip(vocabulary, range(len(vocabulary)))}
vocabulary_by_index = { indice: tipo for (tipo, indice) in zip(vocabulary, range(len(vocabulary)))}

print("index of word 'lugar':  ", vocabulary_by_type['lugar'])
print("word in index 3:        ", vocabulary_by_index[3])

index of word 'lugar':   3
word in index 3:         lugar


### Indexar tokens del corpus

In [307]:
def word_to_index(corpus: list[list[str]], vocab_by_type) -> list[list[int]]:
    """Function that maps each word in a corpus to a unique index"""
    return [ [vocab_by_type[token] for token in sent] for sent in corpus]

In [308]:
indexed_corpus_sentences = word_to_index(tokenized_corpus_sentences, vocabulary_by_type)

In [309]:
print(indexed_corpus_sentences[0])

[0, 1, 2, 3, 4, 5, 6, 7, 4, 8, 9, 10, 11, 12, 13, 7, 10, 14, 15, 16, 17, 18, 19, 2, 20, 4, 21, 4, 22, 1, 23, 24, 25, 26, 7, 27, 28, 29, 30, 31, 7, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]


## Dividir en conjuntos de entrenamiento y prueba

In [310]:
from sklearn.model_selection import train_test_split

corpus_train, corpus_test = train_test_split(indexed_corpus_sentences, test_size=0.3)

len(corpus_train) + len(corpus_test) == len(indexed_corpus_sentences)

print("Train len:", len(corpus_train), "test len:", len(corpus_test))

Train len: 6473 test len: 2775


## Estimación del modelo de n-gramas

In [327]:
import numpy as np
from itertools import chain
from collections import defaultdict

def get_n_grams(indexed_sents: list[list[str]], n=2) -> chain:
    return chain(*[zip(*[sent[i:] for i in range(n)]) for sent in indexed_sents])

def get_model(sents: list[list[str]], vocabulary: defaultdict, n: int=2, l: float=1.0) -> tuple:
    BOS_IDX = vocabulary_by_type['<s>']
    EOS_IDX = vocabulary_by_type['</s>']

    # Get n_grams
    n_grams = get_n_grams(sents, n)

    # Get n_grams frequencies
    freq_n_grams = Counter(n_grams)

    # Get vocabulary length (WITH BOS/EOS)
    N = len(vocabulary)
    # Calculate tensor dimentions for transition probabilities
    dim = (N,)*(n-1) + (N,)

    # Transition tensor
    A = np.zeros(dim)
    # Initial Probabilities
    Pi = np.zeros(N)

    for n_gram, frec in freq_n_grams.items():
        # Fill the tensor with frequencies
        if n_gram[0] != BOS_IDX:
            A[n_gram] = frec
        # Getting initial frequencies
        elif n_gram[0] == BOS_IDX and n_gram[1] != EOS_IDX:
            Pi[n_gram[1]] = frec

    # Calculating probabilities from frequencies
    # We consider the parameter `l` for Lidstone Smoothing
    for h, b in enumerate(A):
        A[h] = ((b+l).T/(b+l).sum(n-2)).T

    # Calculating initial probabilities
    Pi = (Pi+l)/(Pi+l).sum(0)

    # We get our model
    return A, Pi

Estimando un modelo de bigramas con  λ=1

In [325]:
vocabulary_by_index[2061]

'cinda'

In [328]:
%time
bigram_model = get_model(corpus_train, vocabulary_by_type, n=2, l=1)

CPU times: total: 0 ns
Wall time: 0 ns


In [329]:
A_bigram = bigram_model[0]
print("Tensor dimention", A_bigram.shape)
print("Suma de probabilidades")
print(A_bigram.sum(1))

Tensor dimention (2062, 2062)
Suma de probabilidades
[1. 1. 1. ... 1. 1. 1.]


Ahora de trigramas:

In [355]:
%%time
trigram_model = get_model(corpus_train, vocabulary_by_type, n=3, l=1)

MemoryError: Unable to allocate 65.3 GiB for an array with shape (2062, 2062, 2062) and data type float64

In [ ]:
A_trigram = trigram_model[0]
print("Tensor dimention", A_trigram.shape)
print("Suma de probabilidades")
print(A_trigram.sum(1))

## Aplicando el modelo

In [344]:
def get_sent_probability(indexed_sentence: str, vocab: defaultdict, model: tuple) -> float:
    A, Pi = model
    # Getting the n from n-grams
    n = len(A.shape)
    first_indexed_word = indexed_sentence[0]
    # Getting initial probability
    try:
        probability = np.log(Pi[first_indexed_word])
    except:
        print(f"[WARN] OOV for word as BOS with index={first_indexed_word}")
        probability = 0.0

    # Getting n-grams of the sentence
    n_grams = get_n_grams([indexed_sentence], n)
    for n_gram in n_grams:
        try:
            probability += np.log(A[n_gram])
        except:
            print(f"[WARN] OOV for n_gram={n_gram}")
            probability += 0.0

    return probability

In [345]:
def predict_next_word(indexed_sentence: list[int], vocab: defaultdict, model: tuple) -> str:
    A, Pi = model
    history = len(A.shape) - 1
    prev_n_gram = tuple(indexed_sentence[-history:])
    probability = get_sent_probability(indexed_sentence, vocab, model)
    next_word = np.argmax(probability + np.log(A[prev_n_gram]))
    return next_word

In [353]:
TEST_SENTENCE = corpus_test[0][:12]
print("Test sentence:", [vocabulary_by_index[i] for i in TEST_SENTENCE])

Test sentence: ['<s>', 'lo', 'mismo', 'se', 'le', 'dijo', 'al', 'padre', 'de', 'zoraida', ',', 'el']


In [354]:
next_word_index = predict_next_word(TEST_SENTENCE, vocabulary_by_type, bigram_model)
vocabulary_by_index[next_word_index]

'que'